# Latest Product Table

- p_id from **1 to 100** already present in  main table, it can be updated and deleted
- p_id from **101 to 200** can be inserted in to main table and after inserting it into main table it could also be deleted


### Libraries Used

In [7]:
import datetime
import os
import shutil

# Main entry point for DataFrame and SQL functionality.
from pyspark.sql import SparkSession
# Start SPARK Session
spark = SparkSession.builder.appName('Basics').getOrCreate()

from pyspark.sql.functions import *
from pyspark.sql.types import *

AttributeError: 'function' object has no attribute 'parquet'

### Read the Main Table

In [2]:
# Read from main_table
Table = spark.read.format('csv').options(
    header=True, inferschema=True).load(
        "/home/bluepi/Downloads/Update/product_info/main table/main_table_new.csv")

# Add record type to main table
Table_new = Table.withColumn('record_type', lit("A"))

# Write in main_table folder
path = "/home/bluepi/Downloads/Update/Updated Product/Latest Product/"
try:
#     Table_new.write.format('csv').save(
#         os.path.join(path, 'main_table'), header=True)
    Table_new.coalesce(1).write.format('csv').save((path), header=True)

except:
#     Table_new.write.mode('overwrite').format('csv').save(
#         os.path.join(path, 'main_table'), header=True)
    shutil.rmtree("/home/bluepi/Downloads/Update/Updated Product/Latest Product")
    Table_new.coalesce(1).write.format('csv').save((path), header=True)


In [3]:
# GFG
class my_dictionary(dict):

    # __init__ function
    def __init__(self):
        self = dict()

    # Function to add key:value
    def add(self, key, value):
        self[key] = value

### Main CODE

In [4]:
# Storing all the "p_id as value" in a dictionary with "date as key", which we have inserted,deleted & updated
total_products_updated = my_dictionary()
total_products_inserted = my_dictionary()
total_products_deleted = my_dictionary()

# Storing all the p_id in a list which we have deleted and inserted
total_products_inserted_list = []
total_products_deleted_list = []

# ###################################################################################################

for i in range(10, 0, -1):

    data_schema = [
        StructField('p_id', IntegerType(), True),
        StructField('p_name', StringType(), True),
        StructField('price', IntegerType(), True),
        StructField('date_timestamp', TimestampType(), True),
        StructField('record_type', StringType(), True)
    ]
    # Schema which we are accepting
    final_struc = StructType(fields=data_schema)

    # In this try-catch block we are only taking the previous day data, but initially there is
    # no previous day, therefore we are taking main table at begining
    try:
#         path1 = os.path.join(
#             "/home/bluepi/Downloads/Update/Updated Product/Latest Product/", "main_table")
        path1 = "/home/bluepi/Downloads/Update/Updated Product/Latest Product/"
        
        mainTable = spark.read.csv(path1, header=True, schema=final_struc)
        
            
    except:
#         path1 = os.path.join(
#             "/home/bluepi/Downloads/Update/Updated Product/Latest Product/", "main_table")
        path1 = "/home/bluepi/Downloads/Update/Updated Product/Latest Product/"
        
        mainTable = spark.read.csv(path1, header=True, schema=final_struc)
        

#     print("\n File read from --")
#     print(path1, end='\n')

    # Address to the product_info folder
    address = "/home/bluepi/Downloads/Update/product_info/"
    previous_day = (datetime.datetime.today() -
                    datetime.timedelta(days=i)).strftime('%d-%m-%Y')
    print("\n\t\ Date ----> "+previous_day+"\n")

    # before_insert = str(mainTable.count())
    print("Main_Table Count ----> " + str(mainTable.count()))

    # Address to the Previous Day folder
    new_address = address + previous_day
#     print("\nNew Address to read the folder ---->"+new_address)

# ###################################################################################################

    # Read the Previous Day folder
    per_day_data = spark.read.format('csv') \
        .options(header=True, inferschema=True) \
        .load(new_address)

    # Get the list of p_id which we have to inserted in the main table
    to_be_inserted = per_day_data.filter("record_type == 'I' ").collect()
    p_id_list_I = [i.p_id for i in to_be_inserted]

    # Since we don't want our Inserted product to be Inserted again
    for i in total_products_inserted_list:
        if i in p_id_list_I:
            print("\n{} is going to be inserted again\n".format(i))
            # Therefore removing already inserted products
            p_id_list_I.remove(i)
    # Storing all the p_id in a list which we have inserted
    total_products_inserted_list.extend(p_id_list_I)

    # Directly append new Inserted products
    mainTable_I_inserted = mainTable.union(
        per_day_data.filter("record_type == 'I' "))

    after_insert = str(mainTable_I_inserted.count())
    total_insert = str(per_day_data.filter("record_type == 'I' ").count())

    print("Total \"Inserted I\"----> " + total_insert)
    print("After Inserting----> " + after_insert)

    # Storing all the "p_id as value" in a dictionary with "date as key", which we have inserted
    total_products_inserted.add(previous_day, p_id_list_I)
    p_id_list_I.sort()
    print(" ".join(map(str, p_id_list_I)))
    print("\n")

# ###################################################################################################

# Created a new DataFrame of records to be updated
    from_per_day_data_U = per_day_data.filter("record_type == 'U' ")

# Get the list of p_id which we have to update taken from dated folders
    from_per_day_data_U_list = from_per_day_data_U.select("p_id").collect()

# List comprehension
    p_id_list_U = [i.p_id for i in from_per_day_data_U_list]
    p_id_list_U.sort()
    print("TO Be Update\"per_day_data\"")
    print(" ".join(map(str, p_id_list_U)))
#     print(p_id_list_U)

    # Storing all the "p_id as value" in a dictionary with "date as key", which we have updated
    total_products_updated.add(previous_day, p_id_list_U)

    total_update = str(len((p_id_list_U)))
    print("Update Count ------>"+total_update)
    print("\n")

# Get the products which we have to update, present in Main Table
    from_mainTable_U = mainTable_I_inserted.filter(
        col('p_id').isin(p_id_list_U))

    # Performed Union operation on the above tables
    mT_and_pDD_union = from_mainTable_U.union(from_per_day_data_U)
#     mT_and_pDD_union.orderBy(mT_and_pDD_union.p_id).show()

    # Performed GroupBy operation on P_ID and take the latest date only
    #    Rename the columns
    for_join_mT_and_pDD = mT_and_pDD_union.groupBy("p_id").agg(
        {"date_timestamp": "max"}).withColumnRenamed("max(date_timestamp)", "date_timestamp_1")
    for_join_mT_and_pDD = for_join_mT_and_pDD.withColumnRenamed(
        "p_id", "p_id_1")
#     for_join_mT_and_pDD.show()

    # Performed Join operation to pick only latest updates only
    joined = mT_and_pDD_union.join(for_join_mT_and_pDD, (
        mT_and_pDD_union.p_id == for_join_mT_and_pDD.p_id_1) & (
            mT_and_pDD_union.date_timestamp == for_join_mT_and_pDD.date_timestamp_1), 'inner')

    joined = joined.select(
        ['p_id', 'p_name', 'price', 'date_timestamp', 'record_type'])
#     joined.count()

    # First remove the p_id from Main_Table which we have to updated
    mainTable_U_updated = mainTable_I_inserted.filter(
        ~col('p_id').isin(p_id_list_U))
#     mainTable_U_updated.orderBy("p_id").count()

    # Then Add the Updated P_ID to the Main_Table
    mainTable_U_updated_new = mainTable_U_updated.union(joined)
#     after_update = str(mainTable_U_updated_new.orderBy("p_id").count())
#     mainTable_U_updated_new.orderBy("p_id").count()

# ###################################################################################################

    # Get the list of p_id which we have to delete
    to_be_deleted = per_day_data.filter("record_type == 'D' ").collect()
    p_id_list_D = [i.p_id for i in to_be_deleted]

    # Since we don't want our deleted product to be deleted again
    for i in total_products_deleted_list:
        if i in p_id_list_D:
            print("\n{} is going to be deleted again\n".format(i))
            # Therefore removing already deleted products
            p_id_list_D.remove(i)
    # Storing all the p_id in a list which we have deleted
    total_products_deleted_list.extend(p_id_list_D)

    # Storing all the "p_id as value" in a dictionary with "date as key", which we have deleted
    total_products_deleted.add(previous_day, p_id_list_D)


#     print("\nList of p_id which we have to deleted taken from \"per_day_data\"")
#     print(p_id_list_D)
    total_deleted = str(len((p_id_list_D)))
    p_id_list_D.sort()
    print("\nTotal Deleted ------>"+str(len((p_id_list_D))))
    print(" ".join(map(str, p_id_list_D)))

    # Remove the deleted p_id from main_table
    mainTable_D_deleted = mainTable_U_updated_new.filter(
        ~col('p_id').isin(p_id_list_D))

    after_delete = str(mainTable_D_deleted.count())
    print("After Deleting Count---->"+after_delete)


# ###################################################################################################

    # This writes the DF in different files becaues of parallism
    t = "main_table"+str(previous_day)
#     print(t,end="\n")
    
    mainTable_new = spark.createDataFrame([], schema=final_struc)
    mainTable_new = mainTable_D_deleted.union(mainTable_new)

    
    mainTable_new.coalesce(1).write.csv(path, mode= 'overwrite', header=True)

    print("\n Main Table Stored ")
    print(os.path.join(path, t))

    print("\n***************************************************************************************\n")
#     input("Press Enter to continue...")





	\ Date ----> 05-04-2020

Main_Table Count ----> 100
Total "Inserted I"----> 10
After Inserting----> 110
101 115 137 149 158 162 173 181 187 196


TO Be Update"per_day_data"
5 7 12 16 18 19 22 22 28 32 35 40 41 44 46 51 54 58 62 67 77 78 86 90 90 105 136 187 187 187
Update Count ------>30



Total Deleted ------>9
15 26 32 52 53 59 75 79 114
After Deleting Count---->104


Py4JJavaError: An error occurred while calling o248.csv.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:198)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:159)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:104)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:102)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:122)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:131)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:127)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.sca

### Products Updated in a day

In [ ]:
l = mT_and_pDD_union.groupBy("p_id").agg({'p_id':'count'}).filter("count(p_id)>1").select('p_id').collect()
l1 = [ i.p_id for i in l]
mT_and_pDD_union.filter( col('p_id').isin(l1) ).orderBy(col('p_id')).show()

### Final Main Table

In [ ]:
mainTable_D_deleted.filter( col('p_id').isin(l1) ).orderBy(col('p_id')).show()

### Total Insertion, Updation & Deletion Day Wise

In [ ]:
import pprint
# prints the formatted representation of PrettyPrinter object
pp = pprint.PrettyPrinter(indent=4)

for i in range(10,0,-1) :
    previous_day = (datetime.datetime.today() - datetime.timedelta(days=i)).strftime('%d-%m-%Y')
    pp.pprint(previous_day)
    print("inserted :\t")
    print(' '.join(map(str, total_products_inserted.get(previous_day))))
#     pp.pprint(total_products_inserted.get(previous_day))
    print("updated :\t")
    print(' '.join(map(str, total_products_updated.get(previous_day))))
#     pp.pprint(total_products_updated.get(previous_day))
    print("deleted :\t")
    print(' '.join(map(str, total_products_deleted.get(previous_day))))
#     pp.pprint(total_products_deleted.get(previous_day))
    print("\n")

### Total Deleted and Inserted Products

In [ ]:
total_products_deleted_list.sort()
print(' '.join(map(str,total_products_deleted_list)))

In [ ]:
total_products_inserted_list.sort()
print(' '.join(map(str,total_products_inserted_list)))